# Feature Development 

Here I will try and build out some fo the features and think about how I process them. 

This is really just a demonstration of manipulating and creating features using pandas.

Not only do we need to create features but also think about what information will be present at time of prediction as well as what strategies to use when we have no information.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Import package
from urllib.parse import quote_plus
import urllib.request
import urllib.error

import json
import calendar
from datetime import datetime, timezone, timedelta
from dateutil.tz import gettz
import sys
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns



import pickle

# For scheduling
import sched
import time

pd.set_option('display.max_columns', None)

from google.cloud import bigquery

import os
import re

# Define service credentials path
service_path = '/Users/danielpayne/Desktop/Data Science/Greyhounds/credentials/current_greyhound_key.json'


from google.oauth2 import service_account
import pandas_gbq as gbq



import itertools
from sklearn import metrics
from pprint import pprint

# Make the client instance
#client = bigquery.Client()

credentials = service_account.Credentials.from_service_account_file(service_path)

#from lifelines import KaplanMeierFitter, CoxPHFitter

import plotly.graph_objects as go
from plotly.subplots import make_subplots


from dateutil.parser import parse

%matplotlib inline

## Feature Builder class object

This is still useful as it does some cleaning on the dataset.

The class object below is someething I developed a while ago to try and build some features into a dataset.

I am using this onnly to call from the BigQuery table as it also cleans out some anomalous data.

In [2]:
class CorrectedFeatureBuilder():
    
    def __init__(self, google_service_account = service_path):
        
        self.google_service_account = google_service_account
        self.credentials = service_account.Credentials.from_service_account_file(self.google_service_account)
        self.initialise()
        
        
    # Method for getting base infromation
    def get_base_table(self, sql = None):
        
        if sql == None:
            
            dog_statement = '''
                    with feature_table AS (
                    SELECT
                    r.meetingId,
                    'Crayford' as trackName,
                    CAST(r.raceDate as date) as race_date,
                    r.raceClass as grade,
                    r.raceDistance as distance,
                    r.raceId,
                    r.raceGoing,
                    DATETIME(r.dateTime) as datetime,
                    d.dogName as name,
                    CASE WHEN d.dogSex = 'b' then 1 else 0 END as is_female,
                    t.dogId,
                    t.resultAdjustedTime as calc_time,
                    t.resultRunTime as race_time,
                    t.resultSectionalTime as section_time,
                    ROUND(t.resultSectionalTime * (t.resultAdjustedTime / t.resultRunTime), 2) as calc_section_time,
                    (t.resultRunTime - t.resultSectionalTime) as remainder_time,
                    ROUND(t.resultAdjustedTime - (t.resultSectionalTime * (t.resultAdjustedTime / t.resultRunTime)),2 ) as calc_remainder_time,
                    t.resultPosition as position,
                    CASE WHEN t.resultPosition = 1 then 1 else 0 end as winner,
                    t.trapNumber as trap,
                    t.resultComment as comment,
                    t.resultDogWeight as weight,
                    t.trainerName,
                    'race' as race_type
                    from gbgb_db.race_details r 
                    inner join gbgb_db.trap_results t on r.raceId = t.raceId
                    inner join gbgb_db.dog_detail d on t.dogId = d.dogId

                    where t.resultRunTime is not null

                    --order by r.dateTime asc, t.trapNumber asc 
                    
                    UNION ALL
                    
                    SELECT
                    tr.meetingId,
                    'Crayford' as trackName,
                    CAST(tr.raceDate as date) as race_date,
                    tr.raceClass as grade,
                    tr.raceDistance as distance,
                    tr.raceId,
                    tr.raceGoing,
                    DATETIME(tr.dateTime) as datetime,
                    td.dogName as name,
                    CASE WHEN td.dogSex = 'b' then 1 else 0 END as is_female,
                    tt.dogId,
                    tt.resultAdjustedTime as calc_time,
                    tt.resultRunTime as race_time,
                    tt.resultSectionalTime as section_time,
                    ROUND(tt.resultSectionalTime * (tt.resultAdjustedTime / tt.resultRunTime), 2) as calc_section_time,
                    (tt.resultRunTime - tt.resultSectionalTime) as remainder_time,
                    ROUND(tt.resultAdjustedTime - (tt.resultSectionalTime * (tt.resultAdjustedTime / tt.resultRunTime)),2 ) as calc_remainder_time,
                    tt.resultPosition as position,
                    CASE WHEN tt.resultPosition = 1 then 1 else 0 end as winner,
                    tt.trapNumber as trap,
                    tt.resultComment as comment,
                    tt.resultDogWeight as weight,
                    tt.trainerName,
                    'trial' as race_type
                    from gbgb_db.trial_race_details tr 
                    inner join gbgb_db.trial_trap_results tt on tr.raceId = tt.raceId
                    inner join gbgb_db.trial_dog_detail td on tt.dogId = td.dogId

                    where tt.resultRunTime is not null)
                    
                    select * from feature_table
                    
                    order by datetime, trap

                    --order by tr.dateTime asc, tr.raceNumber asc;
                    '''
        else:
            
            dog_statement = sql
        
        
        base_table = gbq.read_gbq(dog_statement, project_id='greyhound-project', 
               credentials=self.credentials)
        
        # Now clean
        base_table = self.base_table_cleaner(base_table)
        
        return base_table
    
    # Method for getting distance averages
    def get_dist_ave(self):
        
        # script
        sql = '''
        with dist_ave AS 

        (SELECT 
        CAST(distance as INT64) as distance, CAST(race_date as date) as race_date,
        ROUND(SUM(calc_time_sum) 
        OVER (
            PARTITION BY distance
            ORDER BY race_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
          ) /

        SUM(calc_time_count)
        OVER (
            PARTITION BY distance
            ORDER BY race_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
          ), 2) AS current_distance_ave

        from (
        SELECT CAST(r.raceDistance as STRING) as distance, 
        r.raceDate as race_date, 
        SUM(t.resultAdjustedTime) as calc_time_sum,
        COUNT(t.resultAdjustedTime) as calc_time_count
        from gbgb_db.race_details r 
        inner join gbgb_db.trap_results t on r.raceId = t.raceId 
        group by 1,2))

        Select *, LAG(current_distance_ave) OVER(PARTITION BY distance ORDER BY race_date) AS ave_dist_time
        from dist_ave
        where distance > 0
        order by 1,2

        '''

        dist_ave_table = gbq.read_gbq(sql, project_id='greyhound-project', 
                       credentials=self.credentials)
        
        return dist_ave_table
    
    # Method for grade distance average
    def get_grade_dist_ave(self):
        
        # script
        sql_g = '''
        with dist_ave AS 

        (SELECT 
        CAST(distance as INT64) as distance, grade, CAST(race_date as date) as race_date,
        ROUND(SUM(calc_time_sum) 
        OVER (
            PARTITION BY distance,grade
            ORDER BY race_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
          ) /

        SUM(calc_time_count)
        OVER (
            PARTITION BY distance, grade
            ORDER BY race_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
          ), 2) AS current_grade_distance_ave

        from (
        SELECT CAST(r.raceDistance as STRING) as distance, r.raceClass as grade,
        r.raceDate as race_date, 
        SUM(t.resultAdjustedTime) as calc_time_sum,
        COUNT(t.resultAdjustedTime) as calc_time_count
        from gbgb_db.race_details r 
        inner join gbgb_db.trap_results t on r.raceId = t.raceId 
        group by 1,2, 3))

        Select *, LAG(current_grade_distance_ave) OVER(PARTITION BY distance, grade ORDER BY race_date) AS grade_dist_ave
        from dist_ave
        where distance > 0
        order by 1,2,3

        '''

        grade_dist_ave_table = gbq.read_gbq(sql_g, project_id='greyhound-project', 
                       credentials=self.credentials)
        
        return grade_dist_ave_table
    
    # Name lookup
    def get_name_lookup(self):
        
        # table
        self.name_lookup_df = gbq.read_gbq('select dogName as bet_name, dogId from gbgb_db.name_lookup',project_id='greyhound-project', 
                       credentials=self.credentials)
        
        # dict
        self.ID_to_name_df = gbq.read_gbq('select dogName as name, dogId from gbgb_db.dog_detail',project_id='greyhound-project', 
                       credentials=self.credentials)
    
    # cleaning ,method
    def base_table_cleaner(self, df):
    
        '''Function toi clean certain values in table'''

        ## Calc time

        # index to change
        too_quick_idx = df.loc[df.calc_time < 22.8].index

        # specifc values
        df.loc[too_quick_idx, ['calc_time', 'race_time']] = \
        df.loc[too_quick_idx, ['calc_time', 'race_time']] * 10

        # change remainder time
        df.loc[too_quick_idx, 'calc_remainder_time'] = \
        df.loc[too_quick_idx, 'calc_time'] - df.loc[too_quick_idx, 'calc_section_time']

        # Next set
        df.loc[df.raceId == 756983, ['calc_time', 'race_time', 'remainder_time', 'calc_remainder_time']] =\
        df.loc[df.raceId == 756983, ['calc_time', 'race_time', 'remainder_time', 'calc_remainder_time']] - 5

        # 380 M
        df.loc[(df.distance == 380) & (df.calc_time > 27), ['calc_time', 'race_time', 'remainder_time', 'calc_remainder_time']] = np.nan
        
        # 540 M
        df.loc[(df.distance == 540) & (df.calc_time > 45), ['calc_time', 'race_time', 'remainder_time', 'calc_remainder_time']] = np.nan

        ## Section Time

        # specific race
        df.loc[df.raceId == 759247, ['section_time', 'calc_section_time']] = \
        df.loc[df.raceId == 759247, ['section_time', 'calc_section_time']] - 10

        # remainder
        df.loc[df.raceId == 759247, ['remainder_time', 'calc_remainder_time']] = \
        df.loc[df.raceId == 759247, ['remainder_time', 'calc_remainder_time']] + 10

        # 380 section times too low
        df.loc[df.section_time < 3, ['section_time', 'calc_section_time','remainder_time', 'calc_remainder_time']] = np.nan

        # 380 Section times too high
        df.loc[(df.distance == 380) & (df.section_time > 5), 
               ['section_time', 'calc_section_time','remainder_time', 'calc_remainder_time']] = np.nan

        # 540m section times - too low
        sec_540_list = ['2022-07-07 16:18:00','2022-04-13 12:09:00','2022-04-13 11:39:00', '2021-03-18 14:42:00',
                  '2022-07-02 20:20:00', '2022-03-19 10:53:00']

        # section
        df.loc[df.datetime.isin(sec_540_list), ['section_time', 'calc_section_time']] = \
        df.loc[df.datetime.isin(sec_540_list), ['section_time', 'calc_section_time']] + 10

        # remainder
        df.loc[df.datetime.isin(sec_540_list), ['remainder_time', 'calc_remainder_time']] = \
        df.loc[df.datetime.isin(sec_540_list), ['remainder_time', 'calc_remainder_time']] + 10

        # 540 section times too high
        df.loc[(df.distance == 540) & (df.section_time > 15), ['section_time', 'calc_section_time', 'remainder_time', 'calc_remainder_time']] = np.nan

        # 740 section times
        df.loc[(df.distance == 714) & 
                ((df.section_time < 22)|(df.section_time > 27)), ['section_time', 'calc_section_time', 'remainder_time', 'calc_remainder_time']] = np.nan

        # return
        return df
    
    # Method to initialise the class object
    def initialise(self):
        
        # Get tables
        self.base_table = self.get_base_table()
        self.dist_ave_table = self.get_dist_ave()
        self.grade_dist_ave_table = self.get_grade_dist_ave()
        
        # Lookup tables
        self.get_name_lookup()
        # Merge table
        self.merge_table  = self.grade_dist_ave_table.merge(self.dist_ave_table, on = ['distance', 'race_date'])
        
        # define merge cols
        self.mrg_cols_dist_ave = ['distance', 'grade', 'race_date', 'grade_dist_ave', 'ave_dist_time']
        
        # clean merge table
        self.merge_table.grade_dist_ave = self.merge_table.grade_dist_ave.fillna(self.merge_table.current_grade_distance_ave + np.random.normal(0, 0.02))
        self.merge_table.grade_dist_ave = self.merge_table.grade_dist_ave.map(lambda x: round(x, 2))
        
        # Make an updater table for last values
        self.updater_table = self.merge_table.groupby(['distance', 'grade'], as_index = False)[['current_grade_distance_ave', 'current_distance_ave']].last()
        
        # clean table
        self.updater_table = self.updater_table.rename(columns = {'current_grade_distance_ave' : 'grade_dist_ave',
                                                                  'current_distance_ave' : 'ave_dist_time'})
        
        print('Object Initialised')
        
    # Method to add enginerred statistical features
    def add_statistical_features(self, bet_table = None):
        
        '''
        Method to add statistical features
        '''
        
        # Check if a bet table is added
        if bet_table.empty:
            
            # Now ave time per dog over distance
            dog_over_dist = self.base_table.groupby(['distance', 'name'])[['calc_time']].expanding().mean().rename(columns = {'calc_time':'dog_ave_dist'}).reset_index().set_index('level_2')
            dog_over_dist['ave_time_per_dog_over_dist'] = dog_over_dist.groupby(['distance','name'])[['dog_ave_dist']].shift()
            
            # Add to table
            self.base_table['ave_time_per_dog_over_dist'] = dog_over_dist['ave_time_per_dog_over_dist']
        
            # Make feature table
            feature_table = self.base_table.merge(self.merge_table[self.mrg_cols_dist_ave], on=['distance','grade','race_date'])
        
            # Make a flag to indicate whether to filter at the end
            self.live_bet = False
            
        else:
            
            # Get unique datetimes to filter on at end
            self.date_list = bet_table.datetime.unique()
            
            # make race_date column
            bet_table['race_date'] = pd.to_datetime(bet_table['date'])
            
            # Add dogID to bet_table
            bet_table = bet_table.merge(self.name_lookup_df, on = 'bet_name', how='left')
            
            # bet name to df
            bet_table = bet_table.merge(self.ID_to_name_df, on = 'dogId', how='left')
            
            # Fill and drop duplicates
            bet_table.name = bet_table.name.fillna(bet_table.bet_name)
            bet_table = bet_table.drop_duplicates(subset=['datetime','trap_no','name'])

            # Concat the two together and take a look
            race_base_df = self.base_table.loc[self.base_table.name.isin(bet_table.name.values) &
                                              (self.base_table.datetime < self.date_list[0])]
            
            # Ave timeover distance
            # Now ave time per dog over distance
            dog_over_dist = race_base_df.groupby(['distance', 'name'])[['calc_time']].expanding().mean().rename(columns = {'calc_time':'dog_ave_dist'}).reset_index().set_index('level_2')
            dog_over_dist['ave_time_per_dog_over_dist'] = dog_over_dist.groupby(['distance','name'])[['dog_ave_dist']].shift()
            
            # Add to table
            race_base_df['ave_time_per_dog_over_dist'] = dog_over_dist['ave_time_per_dog_over_dist']
            
            # Join onto times
            race_base_df = race_base_df.merge(self.merge_table[self.mrg_cols_dist_ave], on=['distance','grade','race_date'])
            
            # Check dates
            if bet_table['race_date'].max() > self.merge_table.race_date.max():
            
                # join ave times onto bet table
                bet_table = bet_table.merge(self.updater_table, on = ['distance','grade'])
            
            else:
                
                # join times on date as well
                bet_table = bet_table.merge(self.merge_table[self.mrg_cols_dist_ave], on=['distance','grade','race_date'])

            # Filter to just the names I'm interested in
            feature_table = pd.concat([race_base_df, bet_table], ignore_index=True) 
            
            # Make a flag to indicate whether to filter at the end
            self.live_bet = True
            
        # quick check
        self.check_table = feature_table
        
        # sort values
        feature_table = feature_table.sort_values(['datetime', 'trap'])
        
        
        #dog_over_dist['ave_time_per_dog_over_dist'] = dog_over_dist['ave_time_per_dog_over_dist'].fillna(dog_over_dist.dog_ave_dist + 0.02)

        # Now ave time per dog per grade over distance
        dog_over_grade_dist = feature_table.groupby(['distance', 'grade','name'])[['calc_time']].expanding().mean().rename(columns = {'calc_time':'grade_dist_ave_time'}).reset_index().set_index('level_3')
        dog_over_grade_dist['grade_dist_ave_time_per_dog'] = dog_over_grade_dist.groupby(['distance', 'grade','name'])[['grade_dist_ave_time']].shift()
        #dog_over_grade_dist.grade_dist_ave_time_per_dog = dog_over_grade_dist.grade_dist_ave_time_per_dog.fillna(dog_over_grade_dist.grade_dist_ave_time + 0.02)


        # Now merge into the feature_table table
        
        #feature_table['ave_time_per_dog_over_dist'] = dog_over_dist['ave_time_per_dog_over_dist']
        feature_table['grade_dist_ave_time_per_dog'] = dog_over_grade_dist['grade_dist_ave_time_per_dog']



        # Create the grouby table and name the second index level
        rolling_3 = self.base_table.groupby(['distance', 'name'])[['calc_time', 'position']].rolling(3, min_periods = 1).mean()
        rolling_3.index = rolling_3.index.rename('index_no', level = 2)


        # Now set the df index to that of the index_no on level 2 of index
        rolling_3 = rolling_3.reset_index().set_index('index_no')
        rolling_3 = rolling_3.rename(columns={'calc_time' : 'rolling_time_3',
                                               'position' : 'position_form_3'})

        # Create the grouby table and name the second index level
        new_exp_mean = self.base_table.groupby(['distance', 'name'])[['calc_time']].expanding().mean()
        new_exp_mean.index = new_exp_mean.index.rename('index_no', level = 2)
        # Now set the df index to that of the index_no on level 2 of index
        new_exp_mean = new_exp_mean.reset_index().set_index('index_no')
        new_exp_mean = new_exp_mean.rename(columns={'calc_time' : 'exp_mean_time'})

        # Create the grouby table and name the second index level
        rolling_5 = self.base_table.groupby(['distance', 'name'])[['calc_time', 'position']].rolling(5, min_periods = 1).mean()
        rolling_5.index = rolling_5.index.rename('index_no', level = 2)
        # Now set the df index to that of the index_no on level 2 of index
        rolling_5 = rolling_5.reset_index().set_index('index_no')
        rolling_5 = rolling_5.rename(columns={'calc_time' : 'rolling_time_5',
                                               'position' : 'position_form_5'})

        # Create the grouby table and name the second index level
        best_5 = self.base_table.groupby(['distance', 'name'])[['calc_time', 'position']].rolling(5, min_periods = 1).min()
        best_5.index = best_5.index.rename('index_no', level = 2)
        # Now set the df index to that of the index_no on level 2 of index
        best_5 = best_5.reset_index().set_index('index_no')
        best_5 = best_5.rename(columns={'calc_time' : 'best_time_5',
                                               'position' : 'best_position_5'})

        # Do the dogs best time and sectional time
        x_best_race_time_table = self.base_table.groupby(['distance','name'])[['calc_time', 'calc_section_time']].expanding().min()
        x_best_race_time_table.index = x_best_race_time_table.index.rename('index_no', level = 2)
        x_best_race_time_table = \
        x_best_race_time_table.reset_index().set_index('index_no').rename(columns = {'calc_time' : 'best_race_time', 
                                                                                     'calc_section_time' : 'best_sectional_time'})

        # Concat the various dfs into one for the merge
        function_df = pd.concat([rolling_3, new_exp_mean.iloc[:,2:], rolling_5.iloc[:,2:], best_5.iloc[:,2:]], axis = 1)

        # Add in extra columns
        function_df['best_race_time'] = x_best_race_time_table.best_race_time
        function_df['best_sectional_time'] = x_best_race_time_table.best_sectional_time

        # make a list fo the columsn for later use
        shift_cols = function_df.columns.tolist()

        # Groupby and shift
        shifted = function_df.groupby(['distance', 'name']).shift()
        
        # Ensure correct merge
        appended_shifted = pd.concat([shifted, self.base_table[['distance', 'name', 'datetime']]], axis=1)

        # Fill the position column nans
        appended_shifted[['best_position_5', 'position_form_3', 'position_form_5']] = appended_shifted[['best_position_5', 'position_form_3', 'position_form_5']].fillna(6.0)



        # The column is how many races the dog has had
        feature_table['race_count'] = feature_table.groupby('name').cumcount() + 1


        # This table is to find the mean race time of dogs by the number of races they've had
        race_count = feature_table.groupby(['distance', 'race_count']).mean()

        # Merge into main df
        feature_table = feature_table.merge(appended_shifted, on = ['distance', 'name', 'datetime'], how='left')

        # fill best time and sectional time nans

        #create groupby table for sectional time
        dist_ave_trap = feature_table.groupby('distance').mean().calc_section_time

        # fill best race time
        feature_table.best_race_time = feature_table.best_race_time.fillna(feature_table.ave_dist_time)

        # now fill sectional time
        feature_table.loc[feature_table.best_sectional_time.isnull(), 'best_sectional_time'] = \
        feature_table.loc[feature_table.best_sectional_time.isnull(), 'distance'].map(lambda x: dist_ave_trap.loc[x])

        # Make a simple loop to fill the nan values
        time_cols = ['exp_mean_time', 'rolling_time_3', 'rolling_time_5']


        for dist in feature_table.distance.unique():
            feature_table.loc[feature_table.distance == dist, 'best_time_5'] = feature_table.loc[feature_table.distance == dist, 'best_time_5'].fillna(feature_table.ave_dist_time)

        # Fill in the rolling times
        feature_table[time_cols] = feature_table[time_cols].fillna(feature_table.calc_time)

        # Make the trend columns
        feature_table['trend_3'] = feature_table.groupby(['distance', 'name'])[['rolling_time_3']].diff()
        feature_table['trend_5'] = feature_table.groupby(['distance', 'name'])[['rolling_time_5']].diff()

        # Fill the nans
        feature_table['trend_3'] = feature_table['trend_3'].fillna(0)
        feature_table['trend_5'] = feature_table['trend_5'].fillna(0)


        # Now can use this to make a column for both pos form 3 and 5
        pos_form_3 = feature_table.groupby('name')[['position']].rolling(3, min_periods = 1).mean()
        feature_table.position_form_3 = pos_form_3.reset_index().set_index('level_1').groupby('name').shift().fillna(6.0)

        pos_form_5 = feature_table.groupby('name')[['position']].rolling(5, min_periods = 1).mean()
        feature_table.position_form_5 = pos_form_5.reset_index().set_index('level_1').groupby('name').shift().fillna(6.0)

        # First I have to create a best time in grade currently.  Will use feature_table df from above
        current_best_time = feature_table.groupby(['distance', 'grade', 'name'])[['calc_time']].rolling(6, min_periods = 1).min()
        feature_table['best_time_in_grade_currently'] = current_best_time.reset_index().set_index('level_3').groupby(['distance', 'grade', 'name']).shift()

        # Now fill the nans
        feature_table['best_time_in_grade_currently'] = feature_table['best_time_in_grade_currently'].fillna(feature_table.grade_dist_ave)

        # make groupby table for race_rating function to reference - this is for the mean times of racing dogs
        mean_best_time = feature_table.groupby(['datetime'])[['best_time_in_grade_currently']].mean()

        # change column name
        mean_best_time = mean_best_time.rename(columns = {'best_time_in_grade_currently' : 'mean_best_time'})

        # create mean best time column
        feature_table = feature_table.merge(mean_best_time, left_on='datetime', right_index=True, how = 'left')

        # Now the race rating column
        feature_table['race_rating'] = feature_table['mean_best_time'] / feature_table.grade_dist_ave

        trend_3_rolling_3 = feature_table.groupby(['distance', 'name'])[['trend_3', 'calc_time']].rolling(3, min_periods = 1).mean()
        trend_3_rolling_3 = trend_3_rolling_3.reset_index()
        trend_3_rolling_3 = trend_3_rolling_3.set_index('level_2')

        feature_table['tr3_rolling_3'] = trend_3_rolling_3.trend_3
        
        # Cleaning
        feature_table.ave_time_per_dog_over_dist = feature_table.ave_time_per_dog_over_dist.fillna(feature_table.ave_dist_time)
        # Change from grade_dist_ave to ave_time_per_dog_over_dist
        feature_table.grade_dist_ave_time_per_dog = \
        feature_table.grade_dist_ave_time_per_dog.fillna(feature_table.ave_time_per_dog_over_dist *(feature_table.grade_dist_ave / feature_table.ave_dist_time))

        feature_table.rolling_time_3 = feature_table.rolling_time_3.fillna(feature_table.grade_dist_ave_time_per_dog)
        feature_table.rolling_time_5 = feature_table.rolling_time_5.fillna(feature_table.grade_dist_ave_time_per_dog)
        feature_table.exp_mean_time = feature_table.exp_mean_time.fillna(feature_table.grade_dist_ave_time_per_dog)
        
        # Month rolling ave
        month_roll = self.base_table.groupby(['distance', 'name'])[['datetime','calc_time']].apply(lambda x: x.set_index('datetime').rolling('28d', min_periods = 1).mean())

        month_roll.calc_time = month_roll.calc_time.map(lambda x: round(x, 2))

        month_roll.columns = ['month_rolling']
        
        month_roll = month_roll.reset_index().drop_duplicates()
        
        s = month_roll.groupby(['distance', 'name'])[['month_rolling']].shift()
        
        month_roll['month_rolling'] = s
        
        feature_table = feature_table.merge(month_roll, on=['distance', 'name', 'datetime'],  how='left')
        
        feature_table.month_rolling = feature_table.month_rolling.fillna(feature_table.grade_dist_ave_time_per_dog)
        

        return feature_table
    
    # Small functiosn to help with win stats
    # Make function to get expected win rate
    def expected_win_rater(self, row):

        if row['winner'] == 1:

            # Get races but only if 4 or above
            races_ = row['race_no']

            if races_ < 6:

                races_ = 6

            est_win_rate = row['cum_wins'] / races_

            return est_win_rate


    # Function for ideal weight
    def race_weight_definer(self, row):

        if (row['weight'] >= row['ideal_weight'] -0.1) and (row['weight'] <= row['ideal_weight'] + 0.1):

            return 1
        else:
            return 0

    # Method to add betting bias
    def add_win_stats(self, df):
        
        '''
        Method to add win stats by dog 
        '''
        
        df['cum_wins'] = df.groupby(['name'])[['winner']].cumsum()
        df['race_no'] = df.groupby('name')[['raceId']].cumcount() + 1

        df['est_win_rate'] = df.apply(self.expected_win_rater, axis=1)
        df.est_win_rate = df.groupby('name')['est_win_rate'].fillna(method = 'ffill')
        df.est_win_rate = df.groupby('name')['est_win_rate'].shift()

        df['current_win_rate'] = df.cum_wins / df.race_no
        df.current_win_rate = df.groupby('name')['current_win_rate'].shift()


        df.est_win_rate = df.est_win_rate.fillna(0.166)
        df.current_win_rate = df.current_win_rate.fillna(0.166)

        df['exp_ratio'] = df.est_win_rate / df.current_win_rate * 0.5
        
        df.exp_ratio = df.exp_ratio.map(lambda x: 0.5 if x == np.inf else x)
        df['win_expected'] = df['exp_ratio'].map(lambda x: 1 if x > 1 else 0)

        top_races = df.groupby(['name', 'distance'], as_index = False).apply(lambda x: x.nsmallest(3, 'calc_time'))
        top_races.weight = top_races.weight.map(lambda x: np.nan if x == 0 else x)
        race_weights = top_races.groupby(['name', 'distance'])[['weight']].median()

        # Change col name and reset index
        race_weights.columns = ['ideal_weight']

        race_weights = race_weights.reset_index()

        
        df = df.merge(race_weights, on=['name', 'distance'], how='left')
       

        df['race_weight'] = df.apply(self.race_weight_definer, axis=1)
        return df
        
        
    # Collection method
    def build_feature_table(self, betting_table = pd.DataFrame()):
        
        '''
        Method to construct feature table
        '''
        
        # feature table
        self.feature_df = self.add_statistical_features(bet_table=betting_table)
        
        # Add win stats
        self.feature_df = self.add_win_stats(self.feature_df)
        
        # Now filter
        if self.live_bet == True:
            
            #filter on date
            self.feature_df = self.feature_df.loc[self.feature_df.datetime.isin(self.date_list)]
        
        return self.feature_df

### Features/Predictors and the Target of Winner


These features were considered as predictors of race finish position. A description of each feature is provided below. Features are divided into relevant groups.

In some cases I have not used Odds and have also used my own naming convention.

These are some ideas I saw to base my features on for this demonstration

**Race identifier**

- 'Race_ID' --- Unique identifier for each race. Use in machine learning questionable.

**Odds related**

- 'Odds'--- Bookmaker odds found in the racing post.
- 'BSP'--- Betfair odds, normally higher than the bookmaker odds.
- 'Public_Estimate'--- 1 to 6 reflecting the expected finish position based on BSP.


**All previous races run by the greyhound**


- 'Races_All'--- Total number of races run by the greyhound prior to the race in question.
- 'Distance_All'--- Average distance for all races run by a greyhound.
- 'Finish_All'--- Average finish position for all races run by a greyhound. Normalised to 6 runner races.
- 'Distance_Places_All'---The average distance of races were the greyhound finished 1st or 2nd. This statistic should indicate a distance preference for each greyhound. For example, if a greyhound has a statistic of 474.0 metres, he appears to perform well at longer distances than the 380.0 metres of the race to be tested.
- 'Wide'--- Percentage of races in which the greyhound ran wide, thereby hampering their win chances. Wide runs are not very common.


**Recent races (seven most recent)**


- 'Distance_Recent'--- Average distance run in seven most recent races.
- 'Finish_Recent'--- Average finish position in seven most recent races. Statistic corrected for trial races.
- 'Odds_Recent'--- Average bookmaker odds seven most recent races.
- 'Early_Recent'--- Average early position in seven most recent races. The recorded position of the greyhound roughly 20 percent of the way into a race.


**Crayford 380 metre races only**

- 'Races_380'--- Total races of this type in form. This is the race type it is now racing in and for which we want to predict an expected finish position.

- 'Wins_380'--- Win percentage for all races at Crayford 380 metres.

- 'Finish_380'--- Average finish position in seven most recent races at Crayford 380 metres. Corrected for trial races.

- 'Odds_380'--- Average bookmaker odds in the seven most recent races at Crayford 380 metres.

- 'Early_380'--- Average relative early position in seven most recent Crayford 380m races.

- 'Grade_380'--- Average race grade in the seven most recent races at Crayford 380 metres.

- 'Stay_380'--- Average finish position minus early position for seven most recent Crayford 380m races. A measure of the greyhound’s stamina at the distance. For example, a statistic of -2.5 indicates that a greyhound starts relatively well but then fails back towards the end.

- 'Time_380'--- Average race completion time for races at Crayford 380 metres. Seven most recent.

- 'Early_Time_380'--- Average time to first bend (20 percent into the race) for races at Crayford 380 metres. Seven most recent.

- 'Wide_380'--- Average number of wide ‘W’ remarks in races at Crayford 380 metres. Seven most recent.

- 'Dist_By'--- Average distance in metres that a greyhound finished to the race winner. Calculated from seven most recent Crayford 380m races.

**Other features**

- 'Trap'--- Trap number from 1 to 6 for each greyhound’s starting trap/stall.
- 'Last_Run'--- Number of days since the greyhound’s last race.
- 'Favourite'--- Trap number of the greyhound who is race favourite i.e. the one with the lowest BSP.

**Target variable**

- 'Finished'--- 1 to 6 for first to sixth finishing places.
or
- 'Winner'--- 1 or 0, for binary win race or not.

## Load data from class object

In [183]:
# Object
builder = CorrectedFeatureBuilder()

# Get base table
base_table = builder.base_table.copy()
print(base_table.shape)
base_table.tail(6)

(156897, 24)


,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type
156891,393972,Crayford,2023-02-11,S6,540.0,913327,0.0,2023-02-11 21:24:00,Hitthelids Holly,1,588350.0,35.14,35.14,13.41,13.41,21.73,21.73,2.0,0,6,"EP,DispTo1,2ndFrom6",25.6,N J Deas,race
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race


In [184]:
# mark open races
base_table['is_open_race'] = base_table.grade.map(lambda x: 1 if x.startswith('OR') else 0)

# Mark out hurdles races
base_table['hurdle_race'] = base_table.grade.map(lambda x: 1 if x.startswith("H") else 0)

# Extract comments
base_table['missed_break'] = base_table.comment.map(lambda x: 1 if pd.notna(x) and (('MsdBrk' in x) or ('MissedBreak' in x)) else 0)
base_table['crowded'] = base_table.comment.map(lambda x: 1 if pd.notna(x) and 'Crd' in x else 0)
base_table['baulked'] = base_table.comment.map(lambda x: 1 if pd.notna(x) and 'Blk' in x else 0)
base_table['bumped'] = base_table.comment.map(lambda x: 1 if pd.notna(x) and 'Bmp' in x else 0)

# Normalise times by distance
normalizer_df = base_table.groupby(['distance', 'hurdle_race'])[['calc_time']].mean().apply(lambda x: round(x, 2))
normalizer_df.columns = ['standard_time']

# Merge onto table
print(base_table.shape)
base_table = base_table.merge(normalizer_df, left_on=['distance', 'hurdle_race'], right_index=True)
print(base_table.shape)

# Base table
base_table['normalised_time'] = round(base_table.calc_time / base_table.standard_time * 100, 2)

# sort index
base_table = base_table.sort_index()
base_table.tail()

(156897, 30)
(156897, 31)


,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15


In [185]:
# Need to make a normalised position.  First get no runners in each race
no_runners = base_table.groupby('datetime')[['name']].count()
no_runners.columns = ['no_runnners']

# join onto main table
print(base_table.shape)
base_table = base_table.merge(no_runners, left_on='datetime', right_index=True)
print(base_table.shape)

base_table['normalised_position'] = base_table['position'] / base_table['no_runnners'] * 6

# section position
base_table['sec_position'] = base_table.groupby('datetime')[['calc_section_time']].rank(method='min')

base_table['normalised_sec_position'] = base_table['sec_position'] / base_table['no_runnners'] * 6
base_table.tail()

(156897, 32)
(156897, 33)


,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6


## Add Features - Name only

These features are based on the individual greyhound and this is the main dimension of these features.

In [186]:
# races all
base_table['races_count'] = base_table.loc[base_table.race_type == 'race'].groupby('name')['race_type'].cumcount() + 1

# measure to aggregate
measure_cols = ['normalised_position', 'distance', 'missed_break', 'crowded', 'baulked', 'bumped']

# concat ave distance and position
dist_pos = base_table.loc[base_table.race_type == 'race'].groupby('name')[measure_cols].expanding().mean().reset_index().set_index('level_1')[measure_cols]

dist_pos.columns = ['ave_position', 'ave_distance', 'ave_missed_break', 'ave_crowded', 'ave_baulked', 'ave_bumped']

print(base_table.shape)
# concat
base_table = pd.concat([base_table, dist_pos], axis=1)

print(base_table.shape)

(156897, 37)
(156897, 43)


In [187]:
# distance all - need to reset and set index for this
base_table['ave_distance_placed'] = base_table.loc[(base_table.race_type == 'race') & (base_table.position <= 2)].groupby('name')[['distance']].expanding().mean().reset_index().set_index('level_1')['distance']


In [188]:
base_table.tail()

,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.180000,380.000000,0.200000,0.466667,0.000000,0.166667,NaN
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.638462,380.000000,0.076923,0.384615,0.025641,0.358974,NaN
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.336842,388.421053,0.210526,0.578947,0.000000,0.421053,380.0
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.633333,380.000000,0.333333,0.666667,0.000000,0.166667,380.0
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.982353,380.000000,0.088235,0.441176,0.000000,0.352941,NaN


### Rolling Features

These features all have an element of a rolling function.

I use aggregation dicts to tidy up the code for doing many aggregations at once.

In [189]:
# define cols to act on
roll_cols = ['distance', 'normalised_position', 'normalised_sec_position', 'normalised_time',
            'missed_break', 'crowded', 'baulked', 'bumped']

# aggregation
agg_dict = {'distance' : 'mean', 'normalised_position' : ['mean', 'std'],
           'normalised_sec_position' : ['mean', 'std'], 'normalised_time' : ['mean', 'min', 'std'],
           'missed_break' : 'mean', 'crowded' : 'mean', 'baulked' : 'mean', 'bumped':'mean'}

df_roll = base_table.loc[base_table.race_type == 'race'].groupby('name')[roll_cols].rolling(7, min_periods=1).agg(agg_dict)

# change cols
df_roll.columns = ['ave_dist_7', 'ave_position_7', 'std_position_7', 'ave_sec_position_7', 'std_sec_position_7',
                  'ave_time_7', 'min_time_7', 'std_time_7', 'msdbrk_7', 'crd_7', 'blk_7', 'bmp_7']

df_roll.index = df_roll.index.get_level_values(1)

In [190]:
# Add to base table
print(base_table.shape)

base_table = pd.concat([base_table, df_roll], axis = 1)
print(base_table.shape)
base_table.tail()

(156897, 44)
(156897, 56)


,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.180000,380.000000,0.200000,0.466667,0.000000,0.166667,NaN,380.0,3.400000,1.754993,3.342857,1.556553,99.461429,97.58,1.182715,0.142857,0.714286,0.0,0.285714
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.638462,380.000000,0.076923,0.384615,0.025641,0.358974,NaN,380.0,2.114286,1.795762,2.114286,1.504913,98.678571,97.66,1.586668,0.000000,0.142857,0.0,0.142857
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.336842,388.421053,0.210526,0.578947,0.000000,0.421053,380.0,380.0,3.200000,1.879716,3.371429,1.714365,99.242857,96.96,1.251262,0.142857,0.571429,0.0,0.571429
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.633333,380.000000,0.333333,0.666667,0.000000,0.166667,380.0,380.0,3.633333,1.965367,2.433333,1.826107,99.453333,97.29,2.080756,0.333333,0.666667,0.0,0.166667
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.982353,380.000000,0.088235,0.441176,0.000000,0.352941,NaN,380.0,2.685714,1.446177,3.100000,1.805547,98.854286,97.08,1.371312,0.000000,0.285714,0.0,0.428571


### Get normalised time but including trial times

Most information is derived from races only.  when greyhounds start their career, having some trial time information can be useful.

In [191]:
# make col
tmp_normal_time = base_table.groupby('name')[['normalised_time']].rolling(7, min_periods=1).agg(['mean', 'min', 'std'])

# cols
tmp_normal_time.columns = ['ave_time_inc_tr_7', 'min_time_inc_tr_7', 'std_time_inc_tr_7']

# index
tmp_normal_time.index = tmp_normal_time.index.get_level_values(1)

# Add to base table
print(base_table.shape)

base_table = pd.concat([base_table, tmp_normal_time], axis = 1)
print(base_table.shape)
base_table.tail()

(156897, 56)
(156897, 59)


,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.180000,380.000000,0.200000,0.466667,0.000000,0.166667,NaN,380.0,3.400000,1.754993,3.342857,1.556553,99.461429,97.58,1.182715,0.142857,0.714286,0.0,0.285714,99.461429,97.58,1.182715
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.638462,380.000000,0.076923,0.384615,0.025641,0.358974,NaN,380.0,2.114286,1.795762,2.114286,1.504913,98.678571,97.66,1.586668,0.000000,0.142857,0.0,0.142857,98.784286,97.66,1.655041
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.336842,388.421053,0.210526,0.578947,0.000000,0.421053,380.0,380.0,3.200000,1.879716,3.371429,1.714365,99.242857,96.96,1.251262,0.142857,0.571429,0.0,0.571429,99.242857,96.96,1.251262
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.633333,380.000000,0.333333,0.666667,0.000000,0.166667,380.0,380.0,3.633333,1.965367,2.433333,1.826107,99.453333,97.29,2.080756,0.333333,0.666667,0.0,0.166667,99.272857,97.29,1.958560
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.982353,380.000000,0.088235,0.441176,0.000000,0.352941,NaN,380.0,2.685714,1.446177,3.100000,1.805547,98.854286,97.08,1.371312,0.000000,0.285714,0.0,0.428571,98.854286,97.08,1.371312


### Add wins and differencing

Will add some stats here to try to inform about how often the greyhound wins as well as some differencing information on the previous times

In [192]:
# Win rate
base_table['win_rate_all_time'] = base_table.loc[base_table.race_type == 'race'].groupby('name')['winner'].expanding().mean().reset_index().set_index('level_1')['winner']

# win rate last 7
base_table['win_rate_last_7'] = base_table.loc[base_table.race_type == 'race'].groupby('name')['winner'].rolling(7, min_periods = 1).mean().reset_index().set_index('level_1')['winner']

# Time diff
base_table['diff_time_last'] = base_table.loc[base_table.race_type == 'race'].groupby('name')['normalised_time'].diff()

base_table['diff_time_last_2'] = base_table.loc[base_table.race_type == 'race'].groupby('name')['normalised_time'].diff(2)

base_table['diff_time_last_shifted'] = base_table.loc[base_table.race_type == 'race'].groupby('name')['diff_time_last'].shift()

base_table.tail()

,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.180000,380.000000,0.200000,0.466667,0.000000,0.166667,NaN,380.0,3.400000,1.754993,3.342857,1.556553,99.461429,97.58,1.182715,0.142857,0.714286,0.0,0.285714,99.461429,97.58,1.182715,0.266667,0.142857,-0.66,1.64,2.30
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.638462,380.000000,0.076923,0.384615,0.025641,0.358974,NaN,380.0,2.114286,1.795762,2.114286,1.504913,98.678571,97.66,1.586668,0.000000,0.142857,0.0,0.142857,98.784286,97.66,1.655041,0.282051,0.571429,4.27,4.39,0.12
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.336842,388.421053,0.210526,0.578947,0.000000,0.421053,380.0,380.0,3.200000,1.879716,3.371429,1.714365,99.242857,96.96,1.251262,0.142857,0.571429,0.0,0.571429,99.242857,96.96,1.251262,0.210526,0.285714,-0.37,-2.01,-1.64
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.633333,380.000000,0.333333,0.666667,0.000000,0.166667,380.0,380.0,3.633333,1.965367,2.433333,1.826107,99.453333,97.29,2.080756,0.333333,0.666667,0.0,0.166667,99.272857,97.29,1.958560,0.166667,0.166667,-3.94,1.89,5.83
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.982353,380.000000,0.088235,0.441176,0.000000,0.352941,NaN,380.0,2.685714,1.446177,3.100000,1.805547,98.854286,97.08,1.371312,0.000000,0.285714,0.0,0.428571,98.854286,97.08,1.371312,0.235294,0.285714,1.77,2.18,0.41


In [193]:
# Stay calculation - depicts whether the greyhound comes back in the race or falls behind after the start
base_table['stay_7'] = base_table.ave_position_7 - base_table.ave_sec_position_7

## Distance related measures

As I am covering all distances, I won't filter to 380m but rather groupby and have the measures for all distances

**Crayford 380 metre races only**

- 'Races_380'--- Total races of this type in form. This is the race type it is now racing in and for which we want to predict an expected finish position.

- 'Wins_380'--- Win percentage for all races at Crayford 380 metres.

- 'Finish_380'--- Average finish position in seven most recent races at Crayford 380 metres. Corrected for trial races.

- 'Odds_380'--- Average bookmaker odds in the seven most recent races at Crayford 380 metres.

- 'Early_380'--- Average relative early position in seven most recent Crayford 380m races.

- 'Grade_380'--- Average race grade in the seven most recent races at Crayford 380 metres.

- 'Stay_380'--- Average finish position minus early position for seven most recent Crayford 380m races. A measure of the greyhound’s stamina at the distance. For example, a statistic of -2.5 indicates that a greyhound starts relatively well but then fails back towards the end.

- 'Time_380'--- Average race completion time for races at Crayford 380 metres. Seven most recent.

- 'Early_Time_380'--- Average time to first bend (20 percent into the race) for races at Crayford 380 metres. Seven most recent.

- 'Wide_380'--- Average number of wide ‘W’ remarks in races at Crayford 380 metres. Seven most recent.

- 'Dist_By'--- Average distance in metres that a greyhound finished to the race winner. Calculated from seven most recent Crayford 380m races.

In [194]:
# Number of races at this distance
base_table['races_by_distance'] = base_table.loc[base_table.race_type == 'race'].groupby(['name', 'distance'])['raceId'].cumcount() + 1

# define cols to act on
roll_cols = ['winner','normalised_position', 'normalised_sec_position', 'normalised_time',
            'missed_break', 'crowded', 'baulked', 'bumped']

# aggregation
agg_dict = {'winner':'mean','normalised_position' : ['mean', 'std'],
           'normalised_sec_position' : ['mean', 'std'], 'normalised_time' : ['mean', 'min', 'std'],
           'missed_break' : 'mean', 'crowded' : 'mean', 'baulked' : 'mean', 'bumped':'mean'}

df_roll = base_table.loc[base_table.race_type == 'race'].groupby(['name', 'distance'])[roll_cols].rolling(7, min_periods=1).agg(agg_dict)

# change cols
df_roll.columns = ['win_pct_distance','ave_dist_position_7', 'std_dist_position_7', 'ave_dist_sec_position_7', 'std_dist_sec_position_7',
                  'ave_dist_time_7', 'min_dist_time_7', 'std_dist_time_7', 'msdbrk_dist_7', 'crd_dist_7', 'blk_dist_7', 'bmp_dist_7']

df_roll.index = df_roll.index.get_level_values(2)

df_roll.head()

,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7
94287,0.0,5.00,NaN,5.00,NaN,101.400000,101.40,NaN,0.0,1.000000,0.0,1.000000
94754,0.0,5.00,0.000000,5.50,0.707107,101.130000,100.86,0.381838,0.0,1.000000,0.0,0.500000
95241,0.0,4.00,1.732051,5.00,1.000000,101.066667,100.86,0.291433,0.0,0.666667,0.0,0.666667
96093,0.0,4.25,1.500000,5.25,0.957427,101.117500,100.86,0.258763,0.0,0.750000,0.0,0.500000
97139,0.0,5.00,NaN,NaN,NaN,102.640000,102.64,NaN,0.0,1.000000,0.0,0.000000


In [195]:
# Add to base table
print(base_table.shape)

base_table = pd.concat([base_table, df_roll], axis = 1)
print(base_table.shape)
base_table.tail()

(156897, 66)
(156897, 78)


,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted,stay_7,races_by_distance,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.180000,380.000000,0.200000,0.466667,0.000000,0.166667,NaN,380.0,3.400000,1.754993,3.342857,1.556553,99.461429,97.58,1.182715,0.142857,0.714286,0.0,0.285714,99.461429,97.58,1.182715,0.266667,0.142857,-0.66,1.64,2.30,0.057143,30.0,0.142857,3.400000,1.754993,3.342857,1.556553,99.461429,97.58,1.182715,0.142857,0.714286,0.0,0.285714
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.638462,380.000000,0.076923,0.384615,0.025641,0.358974,NaN,380.0,2.114286,1.795762,2.114286,1.504913,98.678571,97.66,1.586668,0.000000,0.142857,0.0,0.142857,98.784286,97.66,1.655041,0.282051,0.571429,4.27,4.39,0.12,0.000000,39.0,0.571429,2.114286,1.795762,2.114286,1.504913,98.678571,97.66,1.586668,0.000000,0.142857,0.0,0.142857
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.336842,388.421053,0.210526,0.578947,0.000000,0.421053,380.0,380.0,3.200000,1.879716,3.371429,1.714365,99.242857,96.96,1.251262,0.142857,0.571429,0.0,0.571429,99.242857,96.96,1.251262,0.210526,0.285714,-0.37,-2.01,-1.64,-0.171429,18.0,0.285714,3.200000,1.879716,3.371429,1.714365,99.242857,96.96,1.251262,0.142857,0.571429,0.0,0.571429
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.633333,380.000000,0.333333,0.666667,0.000000,0.166667,380.0,380.0,3.633333,1.965367,2.433333,1.826107,99.453333,97.29,2.080756,0.333333,0.666667,0.0,0.166667,99.272857,97.29,1.958560,0.166667,0.166667,-3.94,1.89,5.83,1.200000,6.0,0.166667,3.633333,1.965367,2.433333,1.826107,99.453333,97.29,2.080756,0.333333,0.666667,0.0,0.166667
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.982353,380.000000,0.088235,0.441176,0.000000,0.352941,NaN,380.0,2.685714,1.446177,3.100000,1.805547,98.854286,97.08,1.371312,0.000000,0.285714,0.0,0.428571,98.854286,97.08,1.371312,0.235294,0.285714,1.77,2.18,0.41,-0.414286,34.0,0.285714,2.685714,1.446177,3.100000,1.805547,98.854286,97.08,1.371312,0.000000,0.285714,0.0,0.428571


In [196]:
# stay calculation but grouped over distance
base_table['stay_at_distance'] = base_table.ave_dist_position_7 - base_table.ave_dist_sec_position_7

## Shift cols accordingly

1. Shifting by Name
2. Shift by Name for trial races
3. Shifting by Name and Distance

In [197]:
# definne name cols
nme_shift_cols = ['ave_position', 'ave_distance', 'ave_missed_break', 'ave_crowded', 'ave_baulked', 'ave_bumped',
                 'ave_distance_placed', 'ave_dist_7', 'ave_position_7', 'std_position_7', 'ave_sec_position_7', 
                  'std_sec_position_7','ave_time_7', 'min_time_7', 'std_time_7', 'msdbrk_7', 'crd_7', 'blk_7',
                  'bmp_7', 'win_rate_all_time', 'win_rate_last_7', 'diff_time_last', 'diff_time_last_2', 
                  'diff_time_last_shifted', 'stay_7']

# trial name
trial_nme_shift_cols = ['ave_time_inc_tr_7', 'min_time_inc_tr_7', 'std_time_inc_tr_7']

# Name & Distance
dst_shift_cols = ['win_pct_distance','ave_dist_position_7', 'std_dist_position_7', 'ave_dist_sec_position_7', 
                  'std_dist_sec_position_7','ave_dist_time_7', 'min_dist_time_7', 'std_dist_time_7', 
                  'msdbrk_dist_7', 'crd_dist_7', 'blk_dist_7', 'bmp_dist_7', 'stay_at_distance']

In [198]:
# Shift operation
base_table[nme_shift_cols] = base_table.loc[base_table.race_type == 'race'].groupby('name')[nme_shift_cols].shift()

# Shift using trial races
base_table[trial_nme_shift_cols] = base_table.groupby('name')[trial_nme_shift_cols].shift()

# Shift operation name and distance
base_table[dst_shift_cols] = base_table.loc[base_table.race_type == 'race'].groupby(['name', 'distance'])[dst_shift_cols].shift()

base_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156897 entries, 0 to 156896
Data columns (total 79 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   meetingId                156897 non-null  int64         
 1   trackName                156897 non-null  object        
 2   race_date                156897 non-null  datetime64[ns]
 3   grade                    156897 non-null  object        
 4   distance                 156897 non-null  float64       
 5   raceId                   156897 non-null  int64         
 6   raceGoing                129532 non-null  float64       
 7   datetime                 156897 non-null  datetime64[ns]
 8   name                     156897 non-null  object        
 9   is_female                156897 non-null  int64         
 10  dogId                    156897 non-null  float64       
 11  calc_time                156798 non-null  float64       
 12  race_time       

### Now do final features

- Days since last race
- races since last win

In [199]:
base_table['days_since_last_race'] = base_table.loc[base_table.race_type == 'race'].groupby(['name'])[['race_date']].diff()
base_table.days_since_last_race = base_table.days_since_last_race.dt.days
base_table.tail()

,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted,stay_7,races_by_distance,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7,stay_at_distance,days_since_last_race
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.165517,380.000000,0.172414,0.448276,0.000000,0.172414,NaN,380.0,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,99.021429,97.41,1.303079,0.275862,0.285714,2.30,1.64,-0.66,0.400000,30.0,0.285714,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,0.400000,11.0
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.550000,380.000000,0.078947,0.368421,0.026316,0.368421,380.0,380.0,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,98.385714,97.66,0.902254,0.289474,0.571429,0.12,0.12,0.00,-0.171429,39.0,0.571429,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,-0.171429,119.0
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.455556,388.888889,0.222222,0.611111,0.000000,0.388889,380.0,380.0,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,99.641429,96.96,1.389609,0.166667,0.142857,-1.64,-0.94,0.70,-0.171429,18.0,0.142857,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,-0.171429,10.0
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.880000,380.000000,0.200000,0.600000,0.000000,0.200000,NaN,380.0,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,99.401429,97.29,1.980862,0.200000,0.200000,5.83,5.13,-0.70,1.440000,6.0,0.200000,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,1.440000,9.0
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.927273,380.000000,0.090909,0.424242,0.000000,0.363636,NaN,380.0,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.428571,98.425714,97.08,0.933003,0.242424,0.285714,0.41,0.65,0.24,-0.380952,34.0,0.285714,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.428571,-0.380952,4.0


In [220]:
base_table['days_since_last_race_tr'] = base_table.groupby(['name'])[['race_date']].diff()
base_table.days_since_last_race_tr = base_table.days_since_last_race_tr.dt.days
base_table.tail()

,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted,stay_7,races_by_distance,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7,stay_at_distance,days_since_last_race,last_win_race_no,races_since_last_win,dist_grade_no,grade_by_distance,days_since_last_race_tr
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.165517,380.000000,0.172414,0.448276,0.000000,0.172414,NaN,380.0,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,99.021429,97.41,1.303079,0.275862,0.285714,2.30,1.64,-0.66,0.400000,30.0,0.285714,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,0.400000,11.0,26.0,4.0,4,3.857143,11.0
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.550000,380.000000,0.078947,0.368421,0.026316,0.368421,380.0,380.0,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,98.385714,97.66,0.902254,0.289474,0.571429,0.12,0.12,0.00,-0.171429,39.0,0.571429,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,-0.171429,119.0,37.0,2.0,4,5.714286,17.0
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.455556,388.888889,0.222222,0.611111,0.000000,0.388889,380.0,380.0,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,99.641429,96.96,1.389609,0.166667,0.142857,-1.64,-0.94,0.70,-0.171429,18.0,0.142857,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,-0.171429,10.0,13.0,6.0,4,3.714286,10.0
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.880000,380.000000,0.200000,0.600000,0.000000,0.200000,NaN,380.0,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,99.401429,97.29,1.980862,0.200000,0.200000,5.83,5.13,-0.70,1.440000,6.0,0.200000,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,1.440000,9.0,4.0,2.0,4,4.800000,9.0
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.927273,380.000000,0.090909,0.424242,0.000000,0.363636,NaN,380.0,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.428571,98.425714,97.08,0.933003,0.242424,0.285714,0.41,0.65,0.24,-0.380952,34.0,0

In [200]:
# Races since last win
winner_table = base_table.loc[(base_table.race_type == 'race') & (base_table.winner == 1), ['races_count']]
winner_table

,races_count
4,1.0
14,1.0
16,1.0
27,1.0
31,1.0
...,...
156871,2.0
156878,5.0
156884,38.0
156890,67.0


In [201]:
# Add col
base_table['last_win_race_no'] = winner_table.races_count

# Shift by dog
base_table['last_win_race_no'] = base_table.loc[base_table.race_type == 'race'].groupby(['name'])['last_win_race_no'].shift()

# Fill forward
base_table['last_win_race_no'] = base_table.loc[base_table.race_type == 'race'].groupby(['name'])['last_win_race_no'].fillna(method = 'ffill')

# Calculate
base_table['races_since_last_win'] = base_table.races_count - base_table.last_win_race_no

base_table.tail()


,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted,stay_7,races_by_distance,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7,stay_at_distance,days_since_last_race,last_win_race_no,races_since_last_win
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.165517,380.000000,0.172414,0.448276,0.000000,0.172414,NaN,380.0,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,99.021429,97.41,1.303079,0.275862,0.285714,2.30,1.64,-0.66,0.400000,30.0,0.285714,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,0.400000,11.0,26.0,4.0
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.550000,380.000000,0.078947,0.368421,0.026316,0.368421,380.0,380.0,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,98.385714,97.66,0.902254,0.289474,0.571429,0.12,0.12,0.00,-0.171429,39.0,0.571429,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,-0.171429,119.0,37.0,2.0
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.455556,388.888889,0.222222,0.611111,0.000000,0.388889,380.0,380.0,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,99.641429,96.96,1.389609,0.166667,0.142857,-1.64,-0.94,0.70,-0.171429,18.0,0.142857,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,-0.171429,10.0,13.0,6.0
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.880000,380.000000,0.200000,0.600000,0.000000,0.200000,NaN,380.0,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,99.401429,97.29,1.980862,0.200000,0.200000,5.83,5.13,-0.70,1.440000,6.0,0.200000,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,1.440000,9.0,4.0,2.0
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.927273,380.000000,0.090909,0.424242,0.000000,0.363636,NaN,380.0,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.428571,98.425714,97.08,0.933003,0.242424,0.285714,0.41,0.65,0.24,-0.380952,34.0,0.285714,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.428571,-0.380952,4.0,32.0,

In [202]:
# make grade number column
base_table['dist_grade_no'] = base_table.grade.map(lambda x: re.search(r'\d+', x).group(0) if re.search(r'\d+', x) is not None else 0)
base_table.tail()

,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted,stay_7,races_by_distance,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7,stay_at_distance,days_since_last_race,last_win_race_no,races_since_last_win,dist_grade_no
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.165517,380.000000,0.172414,0.448276,0.000000,0.172414,NaN,380.0,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,99.021429,97.41,1.303079,0.275862,0.285714,2.30,1.64,-0.66,0.400000,30.0,0.285714,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,0.400000,11.0,26.0,4.0,4
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.550000,380.000000,0.078947,0.368421,0.026316,0.368421,380.0,380.0,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,98.385714,97.66,0.902254,0.289474,0.571429,0.12,0.12,0.00,-0.171429,39.0,0.571429,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,-0.171429,119.0,37.0,2.0,4
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.455556,388.888889,0.222222,0.611111,0.000000,0.388889,380.0,380.0,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,99.641429,96.96,1.389609,0.166667,0.142857,-1.64,-0.94,0.70,-0.171429,18.0,0.142857,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,-0.171429,10.0,13.0,6.0,4
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.880000,380.000000,0.200000,0.600000,0.000000,0.200000,NaN,380.0,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,99.401429,97.29,1.980862,0.200000,0.200000,5.83,5.13,-0.70,1.440000,6.0,0.200000,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,1.440000,9.0,4.0,2.0,4
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.927273,380.000000,0.090909,0.424242,0.000000,0.363636,NaN,380.0,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.428571,98.425714,97.08,0.933003,0.242424,0.285714,0.41,0.65,0.24,-0.380952,34.0,0.285714,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.4285

In [203]:
# Make df
s = base_table.loc[(base_table.race_type == 'race') & (base_table.is_open_race == 0)].groupby(['name', 'distance'])[['dist_grade_no']].rolling(7, min_periods=1).mean()
# shift
s = s.groupby(['name', 'distance']).shift()

# set index
s.index = s.index.get_level_values(2)

# add to df
base_table['grade_by_distance'] = s
base_table.tail()

,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted,stay_7,races_by_distance,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7,stay_at_distance,days_since_last_race,last_win_race_no,races_since_last_win,dist_grade_no,grade_by_distance
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.165517,380.000000,0.172414,0.448276,0.000000,0.172414,NaN,380.0,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,99.021429,97.41,1.303079,0.275862,0.285714,2.30,1.64,-0.66,0.400000,30.0,0.285714,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,0.400000,11.0,26.0,4.0,4,3.857143
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.550000,380.000000,0.078947,0.368421,0.026316,0.368421,380.0,380.0,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,98.385714,97.66,0.902254,0.289474,0.571429,0.12,0.12,0.00,-0.171429,39.0,0.571429,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,-0.171429,119.0,37.0,2.0,4,5.714286
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.455556,388.888889,0.222222,0.611111,0.000000,0.388889,380.0,380.0,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,99.641429,96.96,1.389609,0.166667,0.142857,-1.64,-0.94,0.70,-0.171429,18.0,0.142857,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,-0.171429,10.0,13.0,6.0,4,3.714286
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.880000,380.000000,0.200000,0.600000,0.000000,0.200000,NaN,380.0,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,99.401429,97.29,1.980862,0.200000,0.200000,5.83,5.13,-0.70,1.440000,6.0,0.200000,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,1.440000,9.0,4.0,2.0,4,4.800000
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.927273,380.000000,0.090909,0.424242,0.000000,0.363636,NaN,380.0,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.428571,98.425714,97.08,0.933003,0.242424,0.285714,0.41,0.65,0.24,-0.380952,34.0,0.285714,2.285714,1.112697,2.666667,1.966384

## Final table preparation and cleaning

1. Ensure all values are filled
2. Ensure 6 runners per race

In [221]:
# First reduce table
race_table = base_table.loc[base_table.race_type == 'race'].copy()

race_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129021 entries, 4 to 156896
Data columns (total 85 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   meetingId                129021 non-null  int64         
 1   trackName                129021 non-null  object        
 2   race_date                129021 non-null  datetime64[ns]
 3   grade                    129021 non-null  object        
 4   distance                 129021 non-null  float64       
 5   raceId                   129021 non-null  int64         
 6   raceGoing                101746 non-null  float64       
 7   datetime                 129021 non-null  datetime64[ns]
 8   name                     129021 non-null  object        
 9   is_female                129021 non-null  int64         
 10  dogId                    129021 non-null  float64       
 11  calc_time                129015 non-null  float64       
 12  race_time       

### Define feature cols

Define these then can work out a filling straegy

In [222]:
# define cols
feature_cols = ['trap', 'distance', 'is_female', 'is_open_race', 'hurdle_race', 'races_count', 'ave_position', 
               'ave_distance', 'ave_missed_break', 'ave_crowded', 'ave_baulked', 'ave_bumped', 'ave_distance_placed',
               'ave_dist_7', 'ave_position_7', 'std_position_7', 'ave_sec_position_7', 'std_sec_position_7',
               'ave_time_7', 'min_time_7', 'std_time_7', 'msdbrk_7', 'crd_7', 'blk_7', 'bmp_7', 'ave_time_inc_tr_7',
               'min_time_inc_tr_7', 'std_time_inc_tr_7', 'win_rate_all_time', 'win_rate_last_7', 'diff_time_last',
               'diff_time_last_2', 'diff_time_last_shifted', 'stay_7', 'races_by_distance', 'win_pct_distance',
               'ave_dist_position_7', 'std_dist_position_7', 'ave_dist_sec_position_7', 'std_dist_sec_position_7',
               'ave_dist_time_7', 'min_dist_time_7', 'std_dist_time_7', 'msdbrk_dist_7', 'crd_dist_7', 'blk_dist_7', 
                'bmp_dist_7', 'stay_at_distance', 'days_since_last_race', 'races_since_last_win', 'dist_grade_no',
               'grade_by_distance']

print(len(feature_cols))

52


### Clean Race table

In [223]:
race_table[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129021 entries, 4 to 156896
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     129021 non-null  object 
 1   distance                 129021 non-null  float64
 2   is_female                129021 non-null  int64  
 3   is_open_race             129021 non-null  int64  
 4   hurdle_race              129021 non-null  int64  
 5   races_count              129021 non-null  float64
 6   ave_position             123618 non-null  float64
 7   ave_distance             123618 non-null  float64
 8   ave_missed_break         123618 non-null  float64
 9   ave_crowded              123618 non-null  float64
 10  ave_baulked              123618 non-null  float64
 11  ave_bumped               123618 non-null  float64
 12  ave_distance_placed      43651 non-null   float64
 13  ave_dist_7               123618 non-null  float64
 14  ave_

In [224]:
# Fill position columns
pos_cols = ['ave_position', 'ave_position_7', 'ave_sec_position_7', 'ave_dist_position_7', 'ave_dist_sec_position_7']

# fill df
race_table[pos_cols] = race_table[pos_cols].fillna(3.5)

In [225]:
# Standard deviation
std_cols = [c for c in race_table.columns if c.startswith('std')]

# fill df
race_table[std_cols] = race_table[std_cols].fillna(0)

In [226]:
tme_cols = [c for c in race_table.columns if ('min' in c and 'time' in c) or ('ave' in c and 'time' in c)]

# fill df
race_table.ave_time_7 = race_table.ave_time_7.fillna(race_table.ave_time_inc_tr_7)
race_table.min_time_7 = race_table.min_time_7.fillna(race_table.min_time_inc_tr_7)
race_table.std_time_7 = race_table.std_time_7.fillna(race_table.std_time_inc_tr_7)
race_table[tme_cols] = race_table[tme_cols].fillna(100)

In [227]:
# Fillers for track accidents
crd_fill, blk_fill, md_brk_fill, bmp_fill = race_table[['ave_crowded', 'ave_baulked', 'ave_missed_break', 'ave_bumped']].mean().values

# Now fill df
race_table[['ave_missed_break', 'msdbrk_7', 'msdbrk_dist_7']] = race_table[['ave_missed_break', 'msdbrk_7', 'msdbrk_dist_7']].fillna(md_brk_fill)
race_table[['ave_crowded', 'crd_7', 'crd_dist_7']] = race_table[['ave_crowded', 'crd_7', 'crd_dist_7']].fillna(crd_fill)
race_table[['ave_baulked', 'blk_7', 'blk_dist_7']] = race_table[['ave_baulked', 'blk_7', 'blk_dist_7']].fillna(blk_fill)
race_table[['ave_bumped', 'bmp_7', 'bmp_dist_7']] = race_table[['ave_bumped', 'bmp_7', 'bmp_dist_7']].fillna(bmp_fill)


In [228]:
# distance cols
race_table.ave_distance = race_table.ave_distance.fillna(race_table.distance)
race_table.ave_dist_7 = race_table.ave_dist_7.fillna(race_table.distance)

In [232]:
# Fill 
race_table.days_since_last_race = race_table.days_since_last_race.fillna(race_table.days_since_last_race_tr)

In [233]:
race_table[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129021 entries, 4 to 156896
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     129021 non-null  object 
 1   distance                 129021 non-null  float64
 2   is_female                129021 non-null  int64  
 3   is_open_race             129021 non-null  int64  
 4   hurdle_race              129021 non-null  int64  
 5   races_count              129021 non-null  float64
 6   ave_position             129021 non-null  float64
 7   ave_distance             129021 non-null  float64
 8   ave_missed_break         129021 non-null  float64
 9   ave_crowded              129021 non-null  float64
 10  ave_baulked              129021 non-null  float64
 11  ave_bumped               129021 non-null  float64
 12  ave_distance_placed      43651 non-null   float64
 13  ave_dist_7               129021 non-null  float64
 14  ave_

In [234]:
# Fill rest
race_table.grade_by_distance = race_table.grade_by_distance.fillna(race_table.dist_grade_no)

# races since last win
race_table.races_since_last_win = race_table.races_since_last_win.fillna(race_table.races_count)

# Fill rest with zero
race_table = race_table.fillna(0)


In [235]:
race_table[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129021 entries, 4 to 156896
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     129021 non-null  object 
 1   distance                 129021 non-null  float64
 2   is_female                129021 non-null  int64  
 3   is_open_race             129021 non-null  int64  
 4   hurdle_race              129021 non-null  int64  
 5   races_count              129021 non-null  float64
 6   ave_position             129021 non-null  float64
 7   ave_distance             129021 non-null  float64
 8   ave_missed_break         129021 non-null  float64
 9   ave_crowded              129021 non-null  float64
 10  ave_baulked              129021 non-null  float64
 11  ave_bumped               129021 non-null  float64
 12  ave_distance_placed      129021 non-null  float64
 13  ave_dist_7               129021 non-null  float64
 14  ave_

## Pad the dataframe

Ensure that 6 runners per race. Due to the fact I will run a Tensorflow model over the data, I need to ensure that there are always 6 runners per race.

Will perform a cross join here to try and pad the data

In [236]:
# DAtetimes
dt_df = race_table[['datetime']].drop_duplicates()
trap_df = race_table[['trap']].drop_duplicates()

# Add key
dt_df['key'] = 0
trap_df['key'] = 0

trap_df

,trap,key
4,1,0
5,2,0
6,3,0
7,4,0
8,5,0
9,6,0


In [237]:
# Now make table to join to
joiner_df = dt_df.merge(trap_df, on='key', how='outer')
print(joiner_df.shape)
joiner_df.tail(6)

(134118, 3)


,datetime,key,trap
134112,2023-02-11 21:40:00,0,1
134113,2023-02-11 21:40:00,0,2
134114,2023-02-11 21:40:00,0,3
134115,2023-02-11 21:40:00,0,4
134116,2023-02-11 21:40:00,0,5
134117,2023-02-11 21:40:00,0,6


In [238]:
race_table.tail()

,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,datetime,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,trap,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted,stay_7,races_by_distance,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7,stay_at_distance,days_since_last_race,last_win_race_no,races_since_last_win,dist_grade_no,grade_by_distance,days_since_last_race_tr
156892,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Always Ella,1,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0,1,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0,0,1,1,0,0,24.35,100.49,5,3.6,5.0,6.0,30.0,3.165517,380.000000,0.172414,0.448276,0.000000,0.172414,0.0,380.0,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,99.021429,97.41,1.303079,0.275862,0.285714,2.30,1.64,-0.66,0.400000,30.0,0.285714,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,0.400000,11.0,26.0,4.0,4,3.857143,11.0
156893,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Icaals Mindy,1,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0,2,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0,0,0,1,0,0,24.35,102.05,5,6.0,4.0,4.8,39.0,2.550000,380.000000,0.078947,0.368421,0.026316,0.368421,380.0,380.0,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,98.385714,97.66,0.902254,0.289474,0.571429,0.12,0.12,0.00,-0.171429,39.0,0.571429,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,-0.171429,119.0,37.0,2.0,4,5.714286,17.0
156894,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Someblake,0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1,3,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0,0,0,0,0,1,24.35,98.44,5,1.2,1.0,1.2,19.0,3.455556,388.888889,0.222222,0.611111,0.000000,0.388889,380.0,380.0,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,99.641429,96.96,1.389609,0.166667,0.142857,-1.64,-0.94,0.70,-0.171429,18.0,0.142857,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,-0.171429,10.0,13.0,6.0,4,3.714286,10.0
156895,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Drumcrow Hydro,0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0,5,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0,0,1,1,0,0,24.35,99.18,5,2.4,2.0,2.4,6.0,3.880000,380.000000,0.200000,0.600000,0.000000,0.200000,0.0,380.0,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,99.401429,97.29,1.980862,0.200000,0.200000,5.83,5.13,-0.70,1.440000,6.0,0.200000,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,1.440000,9.0,4.0,2.0,4,4.8,9.0
156896,393972,Crayford,2023-02-11,A4,380.0,913328,0.0,2023-02-11 21:40:00,Black Lynx,1,599848.0,24.63,24.63,3.64,3.64,20.99,20.99,4.0,0,6,"Crd&Wide 1/4,Crd2",29.6,J Turner,race,0,0,0,1,0,0,24.35,101.15,5,4.8,3.0,3.6,34.0,2.927273,380.000000,0.090909,0.424242,0.000000,0.363636,0.0,380.0,2.285714,1.112697,2.666667,1.966384,98.425714,97.08,0.933003,0.000000,0.142857,0.0,0.428571,98.425714,97.08,0.933003,0.242424,0.285714,0.41,0.65,0.24,-0.380952,34.0,0.2857

In [272]:
# drop key
#joiner_df = joiner_df.drop('key', 1)

# padded df
padded_df = joiner_df.merge(race_table, on = ['datetime', 'trap'], how='left')
print(padded_df.shape, joiner_df.shape, race_table.shape)
padded_df.tail(6)

(134118, 85) (134118, 2) (129021, 85)


,datetime,trap,meetingId,trackName,race_date,grade,distance,raceId,raceGoing,name,is_female,dogId,calc_time,race_time,section_time,calc_section_time,remainder_time,calc_remainder_time,position,winner,comment,weight,trainerName,race_type,is_open_race,hurdle_race,missed_break,crowded,baulked,bumped,standard_time,normalised_time,no_runnners,normalised_position,sec_position,normalised_sec_position,races_count,ave_position,ave_distance,ave_missed_break,ave_crowded,ave_baulked,ave_bumped,ave_distance_placed,ave_dist_7,ave_position_7,std_position_7,ave_sec_position_7,std_sec_position_7,ave_time_7,min_time_7,std_time_7,msdbrk_7,crd_7,blk_7,bmp_7,ave_time_inc_tr_7,min_time_inc_tr_7,std_time_inc_tr_7,win_rate_all_time,win_rate_last_7,diff_time_last,diff_time_last_2,diff_time_last_shifted,stay_7,races_by_distance,win_pct_distance,ave_dist_position_7,std_dist_position_7,ave_dist_sec_position_7,std_dist_sec_position_7,ave_dist_time_7,min_dist_time_7,std_dist_time_7,msdbrk_dist_7,crd_dist_7,blk_dist_7,bmp_dist_7,stay_at_distance,days_since_last_race,last_win_race_no,races_since_last_win,dist_grade_no,grade_by_distance,days_since_last_race_tr
134112,2023-02-11 21:40:00,1,393972.0,Crayford,2023-02-11,A4,380.0,913328.0,0.0,Always Ella,1.0,606913.0,24.47,24.47,3.70,3.70,20.77,20.77,3.0,0.0,"MsdBrk,BCrd 1/4,Rls",27.7,T M Levers,race,0.0,0.0,1.0,1.0,0.0,0.0,24.35,100.49,5.0,3.6,5.0,6.0,30.0,3.165517,380.000000,0.172414,0.448276,0.000000,0.172414,0.0,380.0,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,99.021429,97.41,1.303079,0.275862,0.285714,2.30,1.64,-0.66,0.400000,30.0,0.285714,3.057143,1.934647,2.657143,1.209486,99.021429,97.41,1.303079,0.000000,0.571429,0.0,0.285714,0.400000,11.0,26.0,4.0,4,3.857143,11.0
134113,2023-02-11 21:40:00,2,393972.0,Crayford,2023-02-11,A4,380.0,913328.0,0.0,Icaals Mindy,1.0,582057.0,24.85,24.85,3.65,3.65,21.20,21.20,5.0,0.0,"EP,MvdOff&Crd 1/4,Wide&Crd2",30.1,J W Reynolds,race,0.0,0.0,0.0,1.0,0.0,0.0,24.35,102.05,5.0,6.0,4.0,4.8,39.0,2.550000,380.000000,0.078947,0.368421,0.026316,0.368421,380.0,380.0,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,98.385714,97.66,0.902254,0.289474,0.571429,0.12,0.12,0.00,-0.171429,39.0,0.571429,1.685714,0.790419,1.857143,1.056499,98.327143,97.66,0.785784,0.000000,0.142857,0.0,0.142857,-0.171429,119.0,37.0,2.0,4,5.714286,17.0
134114,2023-02-11 21:40:00,3,393972.0,Crayford,2023-02-11,A4,380.0,913328.0,0.0,Someblake,0.0,609291.0,23.97,23.97,3.60,3.60,20.37,20.37,1.0,1.0,"EP,SnLdTo 1/2,Bmp&Ld3",32.7,J Turner,race,0.0,0.0,0.0,0.0,0.0,1.0,24.35,98.44,5.0,1.2,1.0,1.2,19.0,3.455556,388.888889,0.222222,0.611111,0.000000,0.388889,380.0,380.0,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,99.641429,96.96,1.389609,0.166667,0.142857,-1.64,-0.94,0.70,-0.171429,18.0,0.142857,3.885714,1.903881,4.057143,1.660178,99.641429,96.96,1.389609,0.142857,0.714286,0.0,0.571429,-0.171429,10.0,13.0,6.0,4,3.714286,10.0
134115,2023-02-11 21:40:00,4,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134116,2023-02-11 21:40:00,5,393972.0,Crayford,2023-02-11,A4,380.0,913328.0,0.0,Drumcrow Hydro,0.0,605430.0,24.15,24.15,3.62,3.62,20.53,20.53,2.0,0.0,"MsdBrk,EP,Ld 1/2-Crd3",32.0,D W Lee,race,0.0,0.0,1.0,1.0,0.0,0.0,24.35,99.18,5.0,2.4,2.0,2.4,6.0,3.880000,380.000000,0.200000,0.600000,0.000000,0.200000,0.0,380.0,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,99.401429,97.29,1.980862,0.200000,0.200000,5.83,5.13,-0.70,1.440000,6.0,0.200000,3.880000,2.090933,2.440000,2.041568,99.508000,97.29,2.321534,0.200000,0.600000,0.0,0.200000,1.440000,9.0,4.0,2.0,4,4.8,9.0
134117,20

In [273]:
padded_df[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134118 entries, 0 to 134117
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     134118 non-null  object 
 1   distance                 129021 non-null  float64
 2   is_female                129021 non-null  float64
 3   is_open_race             129021 non-null  float64
 4   hurdle_race              129021 non-null  float64
 5   races_count              129021 non-null  float64
 6   ave_position             129021 non-null  float64
 7   ave_distance             129021 non-null  float64
 8   ave_missed_break         129021 non-null  float64
 9   ave_crowded              129021 non-null  float64
 10  ave_baulked              129021 non-null  float64
 11  ave_bumped               129021 non-null  float64
 12  ave_distance_placed      129021 non-null  float64
 13  ave_dist_7               129021 non-null  float64
 14  ave_

### Cleaning

DEvelpo strategies fro imputing / cleaning each feature

In [274]:
# FEatures which should be the same as others
consistent_race_features = ['distance', 'is_open_race', 'hurdle_race', 'dist_grade_no', 'meetingId',
                           'trackName', 'race_date', 'grade']

# Fill
padded_df[consistent_race_features] = padded_df.groupby('datetime')[consistent_race_features].fillna(method = 'ffill').fillna(method='bfill')

# check
padded_df[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134118 entries, 0 to 134117
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     134118 non-null  object 
 1   distance                 134118 non-null  float64
 2   is_female                129021 non-null  float64
 3   is_open_race             134118 non-null  float64
 4   hurdle_race              134118 non-null  float64
 5   races_count              129021 non-null  float64
 6   ave_position             129021 non-null  float64
 7   ave_distance             129021 non-null  float64
 8   ave_missed_break         129021 non-null  float64
 9   ave_crowded              129021 non-null  float64
 10  ave_baulked              129021 non-null  float64
 11  ave_bumped               129021 non-null  float64
 12  ave_distance_placed      129021 non-null  float64
 13  ave_dist_7               129021 non-null  float64
 14  ave_

#### Straight fills

Fill with an arbitary value like -1

In [268]:
# Make list
neg_fills = ['is_female', 'races_count', 'ave_distance', 'ave_distance_placed', 'ave_dist_7', 'ave_sec_position_7',
            'races_by_distance', 'days_since_last_race', 'races_since_last_win']

padded_df[neg_fills] = padded_df[neg_fills].fillna(-1)

# check
padded_df[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134118 entries, 0 to 134117
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     134118 non-null  object 
 1   distance                 134118 non-null  float64
 2   is_female                134118 non-null  float64
 3   is_open_race             134118 non-null  float64
 4   hurdle_race              134118 non-null  float64
 5   races_count              134118 non-null  float64
 6   ave_position             129021 non-null  float64
 7   ave_distance             134118 non-null  float64
 8   ave_missed_break         129021 non-null  float64
 9   ave_crowded              129021 non-null  float64
 10  ave_baulked              129021 non-null  float64
 11  ave_bumped               129021 non-null  float64
 12  ave_distance_placed      134118 non-null  float64
 13  ave_dist_7               134118 non-null  float64
 14  ave_

In [275]:
# Make list
zero_fills = ['std_position_7', 'std_sec_position_7', 'std_time_7', 'std_time_inc_tr_7', 'win_rate_all_time',
              'win_rate_last_7','diff_time_last', 'diff_time_last_2', 'diff_time_last_shifted', 'stay_7',
              'win_pct_distance','std_dist_position_7', 'std_dist_sec_position_7', 'std_dist_time_7',
             'stay_at_distance']

padded_df[zero_fills] = padded_df[zero_fills].fillna(0)

# check
padded_df[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134118 entries, 0 to 134117
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     134118 non-null  object 
 1   distance                 134118 non-null  float64
 2   is_female                129021 non-null  float64
 3   is_open_race             134118 non-null  float64
 4   hurdle_race              134118 non-null  float64
 5   races_count              129021 non-null  float64
 6   ave_position             129021 non-null  float64
 7   ave_distance             129021 non-null  float64
 8   ave_missed_break         129021 non-null  float64
 9   ave_crowded              129021 non-null  float64
 10  ave_baulked              129021 non-null  float64
 11  ave_bumped               129021 non-null  float64
 12  ave_distance_placed      129021 non-null  float64
 13  ave_dist_7               129021 non-null  float64
 14  ave_

### Will fill the rest with negative 1 as these values can't be attained.

In [276]:
# first fill winner column
padded_df.winner = padded_df.winner.fillna(0)

In [277]:
# Now fill the rest of features with -1
padded_df[feature_cols] = padded_df[feature_cols].fillna(-1)

# check
padded_df[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134118 entries, 0 to 134117
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     134118 non-null  object 
 1   distance                 134118 non-null  float64
 2   is_female                134118 non-null  float64
 3   is_open_race             134118 non-null  float64
 4   hurdle_race              134118 non-null  float64
 5   races_count              134118 non-null  float64
 6   ave_position             134118 non-null  float64
 7   ave_distance             134118 non-null  float64
 8   ave_missed_break         134118 non-null  float64
 9   ave_crowded              134118 non-null  float64
 10  ave_baulked              134118 non-null  float64
 11  ave_bumped               134118 non-null  float64
 12  ave_distance_placed      134118 non-null  float64
 13  ave_dist_7               134118 non-null  float64
 14  ave_

In [278]:
padded_df[['trap', 'dist_grade_no', 'grade_by_distance']] = padded_df[['trap', 'dist_grade_no', 'grade_by_distance']].astype(float)

In [279]:
# check
padded_df[feature_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134118 entries, 0 to 134117
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trap                     134118 non-null  float64
 1   distance                 134118 non-null  float64
 2   is_female                134118 non-null  float64
 3   is_open_race             134118 non-null  float64
 4   hurdle_race              134118 non-null  float64
 5   races_count              134118 non-null  float64
 6   ave_position             134118 non-null  float64
 7   ave_distance             134118 non-null  float64
 8   ave_missed_break         134118 non-null  float64
 9   ave_crowded              134118 non-null  float64
 10  ave_baulked              134118 non-null  float64
 11  ave_bumped               134118 non-null  float64
 12  ave_distance_placed      134118 non-null  float64
 13  ave_dist_7               134118 non-null  float64
 14  ave_

In [280]:
import pickle

In [281]:
# Save df
pickle.dump(padded_df, open( "padded_greyhound.pkl", "wb" ))

In [283]:
# Make smaller dataframe as large file
session_df = padded_df.loc[(padded_df.datetime >= '2020-05-01')].copy()

In [284]:
# Save to CSV
session_df.to_csv('padded_greyhound.csv', index = False)